In [33]:
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [34]:
df = pd.read_csv("C:\dev\datasets\iot23_combined.csv")

In [35]:
df

,Unnamed: 0,ts,id.orig_h,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,1.540470e+09,192.168.1.132,0.003497,0,0,0.0,5.0,212.0,3.0,144.0,Benign,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,1.540470e+09,192.168.1.132,0.036724,34,311,0.0,1.0,62.0,1.0,339.0,Benign,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,1.540469e+09,192.168.1.132,384.518261,15072,0,0.0,48.0,16416.0,0.0,0.0,Benign,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,1.540470e+09,192.168.1.132,0.270332,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,4,1.540470e+09,192.168.1.132,0.111429,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,99994,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1446617,99995,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1446618,99996,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1446619,99997,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [36]:
df['label'].value_counts()

PartOfAHorizontalPortScan     825939
Okiru                         262690
-   Benign   -                146275
DDoS                          138777
Benign                         53481
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: label, dtype: int64

In [37]:
df['id.orig_h'].value_counts()

192.168.1.197      204393
192.168.1.198      199987
192.168.100.111    199935
192.168.1.195      123106
192.168.100.103    100647
                    ...  
87.140.41.9             1
221.135.134.4           1
103.5.159.28            1
69.108.2.131            1
200.170.196.189         1
Name: id.orig_h, Length: 3296, dtype: int64

In [38]:
df_node = df[df['id.orig_h']=='192.168.1.198']

In [39]:
df_node.shape

(199987, 28)

In [40]:
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
-   Benign   -                30016
C&C-HeartBeat                   114
C&C                              29
C&C-FileDownload                 13
Name: label, dtype: int64

In [41]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [42]:
df_node = df_node.drop(columns=['Unnamed: 0', 'id.orig_h'])

In [47]:
df_node['ts']

531573    1.545396e+09
531574    1.545396e+09
531575    1.545396e+09
531576    1.545396e+09
531577    1.545396e+09
              ...     
835988    1.547145e+09
835989    1.547145e+09
835990    1.547145e+09
835991    1.547145e+09
835992    1.547145e+09
Name: ts, Length: 199987, dtype: float64

In [15]:
x = df_node.loc[:, df_node.columns != 'label'].to_numpy()
df_y = df_node.loc[:, df_node.columns == 'label']
df_y = pd.get_dummies(df_y, columns=['label'])
y = df_y.to_numpy()

In [26]:
scaler = MinMaxScaler()

x_scaled = scaler.fit_transform(x)

In [16]:
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
Benign                        30016
C&C-HeartBeat                   114
C&C                              29
C&C-FileDownload                 13
Name: label, dtype: int64

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=42)

In [118]:
model = Sequential()
model.add(Dense(12, input_dim=x.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
gru_model = Sequential()
# gru_model.add(Dense(x.shape[1], activation='relu'))
gru_model.add(GRU(units=128))
# gru_model.add(GRU(units=128))
# gru_model.add(GRU(units=128))
gru_model.add(Dense(y.shape[1], activation='sigmoid'))

gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [119]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
 77030/159989 [=============>................] - ETA: 8s - loss: 0.9820 - acc: 0.5634

KeyboardInterrupt: 

In [27]:
x_train2 = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))

In [29]:
gru_model.fit(x_train2, y_train, epochs=10, batch_size=10)

SyntaxError: invalid syntax (Temp/ipykernel_28000/1668875231.py, line 2)

In [103]:
preds = model.predict_classes(x_test)
num_correct = 0

for i in range(0, len(preds)):
    if preds[i] == np.argmax(y_test[i]):
        num_correct+=1

num_correct/len(preds)

0.15055752787639382

In [64]:
model.save("/Users/kjwdamme/PythonProjects/datasets/first_model_scaled.h5")

In [43]:
preds.shape

(39998,)

In [46]:
np.unique(preds, return_counts=True)

(array([0, 2, 3, 4]), array([    2,     2,    17, 39977]))

In [51]:
temp = np.transpose(y_test)


(6, 39998)

In [54]:
temp_df = pd.DataFrame(y_test)

In [62]:
temp_df[4].value_counts()

1    22662
0    17336
Name: 4, dtype: int64